# scRNASeq analysis

# Loading data

In [ ]:
write_plots <- TRUE
plot_dir <- "" 

In [15]:
theme_set(theme_bw(base_size = 18) +
          theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank()))

In [3]:
suppressWarnings(suppressMessages(library(Seurat)))
suppressWarnings(suppressMessages(library(tidyverse)))
suppressWarnings(suppressMessages(library(cowplot)))
suppressWarnings(suppressMessages(library(patchwork)))
suppressWarnings(suppressMessages(library(readxl)))
suppressWarnings(suppressMessages(library(cluster)))
# suppressWarnings(suppressMessages(library(harmony)))

In [ ]:
manifest <- read.csv("Data/scRNA/kidney/mouse_NTN/manifest_snRNAseq.csv")

In [ ]:
normalized_seurat <- readRDS("/Data/scRNA/kidney/mouse_NTN/processed/annotated_v2.rds")

In [ ]:
df <- manifest %>% select(alias_id, fw_sample_id, Treatment) %>% arrange(alias_id)
df
if (write_plots) {
    write.csv(df, paste0(plot_dir,"/dfs/scRNAseq_samples_Treatment.csv"), row.names = F)
}

# QC

In [40]:
normalized_seurat$Treatment <- factor(normalized_seurat$Treatment, levels = c("vehicle", "Sema", "Enalapril", "Healthy"))

In [ ]:
options(repr.plot.width = 10, repr.plot.height =12)
p_counts <- ggplot(normalized_seurat@meta.data, aes(x = as.factor(alias_id), y = nCount_RNA, fill = as.factor(Treatment))) +
    geom_violin() +
    geom_boxplot(width = 0.1) +
    theme(legend.position = "none",
         axis.title.x=element_blank(),
         axis.text.x=element_blank(),
            axis.ticks.x=element_blank()) +
    ylab('Number Counts')

p_genes <- ggplot(normalized_seurat@meta.data, aes(x = as.factor(alias_id), y = nFeature_RNA, fill = as.factor(Treatment))) +
    geom_violin() +
    geom_boxplot(width = 0.1) +
    theme(
        legend.position = "none",
         axis.title.x=element_blank(),
        axis.text.x=element_blank(),
        axis.ticks.x=element_blank()) +
    ylab('Number Genes') +
    guides(fill=guide_legend(title="Treatment"))

p_mito <- ggplot(normalized_seurat@meta.data, aes(x = as.factor(alias_id), y = percent_mito, fill = as.factor(Treatment))) +
    geom_violin() +
    geom_boxplot(width = 0.1) +
    theme(
        legend.position = "bottom",
         axis.title.x=element_blank(),
        axis.text.x=element_blank(),
        axis.ticks.x=element_blank()) +
    ylab('Mitochondrial %') +
    xlab("Sample id") +
    guides(fill=guide_legend(title="Treatment"))

p <- wrap_plots(p_counts, p_genes, p_mito, ncol = 1)

plot(p)

if(write_plots){
    ggsave(p, filename = file.path(plot_dir, "/plots/QC_scRNAseq/Count_features_mito_violin.png"), width = 10, height = 12)
}

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 7)

plot_list <- lapply(c("alias_id", "Treatment"), FUN = function(f) {
    DimPlot(normalized_seurat, reduction = "pca", group.by = f, pt.size = 0.5) + theme(legend.position = "top")
})

p <- wrap_plots(plot_list, ncol = 2)

plot(p)

if (write_plots) {
    ggsave(p, filename = file.path(plot_dir, "/plots/QC_scRNAseq/integrated_batch_pca.png"), width = 16, height = 7)
}

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 7)

plot_list <- lapply(c("alias_id", "Treatment"), FUN = function(f) {
    DimPlot(normalized_seurat, reduction = "harmony", group.by = f, pt.size = 0.5) + theme(legend.position = "top")
})

p <- wrap_plots(plot_list, ncol = 2)

plot(p)

if (write_plots) {
    ggsave(p, filename = file.path(plot_dir, "/plots/QC_scRNAseq/integrated_batch_harmony.png"), width = 12, height = 7)
}

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 7)

plot_list <- lapply(c("alias_id", "Treatment"), FUN = function(f) {
    DimPlot(normalized_seurat, reduction = "umap", group.by = f, pt.size = 0.5) + theme(legend.position = "top")
})

p <- wrap_plots(plot_list, ncol = 2)

plot(p)

if (write_plots) {
    ggsave(p, filename = file.path(plot_dir, "/plots/QC_scRNAseq/integrated_batch_umap.png"), width = 12, height = 7)
}

In [ ]:
# after analysis
normalized_seurat <- readRDS("./Data/scRNA/kidney/mouse_NTN/processed/normalized_v2.rds")

## Well-known Marker genes 

https://www.nature.com/articles/s41467-021-22266-1

In [ ]:
# Specify the path to your Excel file
file_path <- "/annotation/Miao.Z_Nature.2021/41467_2021_22266_MOESM5_ESM.xlsx"

# Read the specific sheet and skip the first row
markers <- read_excel(file_path, sheet = "Marker genes for annotation", skip=1, col_names = TRUE)
markers_long <- pivot_longer(markers, everything(), names_to = "cell_type", values_to="marker")

In [91]:
all_markers <- unique(unlist(markers))

all_markers <- all_markers[all_markers %in% rownames(normalized_seurat)]

p <- DotPlot(normalized_seurat, features = all_markers)
df <- p$data

df$cell_type <-  markers_long[match(df$features.plot, markers_long$marker), "cell_type"] %>% pull()
df <- df %>% arrange(cell_type)

In [92]:
dotplot <- df %>%  
  mutate(`% Expressed` = pct.exp) %>% 
    mutate(`Avg expressed` = avg.exp.scaled	) %>%
#remove expression expression of 0 or less 20%
  filter(`Avg expressed` > 0, `% Expressed` > 15) %>% 
  ggplot(aes(x=features.plot, y = id, color = `Avg expressed`, size = `% Expressed`)) + 
  geom_point() +
  cowplot::theme_cowplot() + 
  theme(axis.line  = element_blank()) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size=11)) +
  theme(axis.ticks = element_blank()) +
  theme(panel.grid.major = element_line(color = "grey",
                                        size = 0.3,
                                        linetype = 2)) +
labs(x = "features", y = "clusters") +
scale_y_discrete(position = "right") 

celltypes <- df %>%
    mutate(`Cell Type` = cell_type) %>%
    ggplot(aes(x = features.plot, y = 1, fill = `Cell Type`)) + 
  geom_tile() +
    theme_nothing()

legend <- plot_grid(get_legend(celltypes + theme(legend.position="bottom")))

Warning message in get_plot_component(plot, "guide-box"):
“Multiple components found; returning the first one. To return all, use `return_all = TRUE`.”


In [ ]:
options(repr.plot.width = 16, repr.plot.height = 10, res=1200)
final_plot <-  celltypes + 
  plot_spacer() +
  dotplot + 
   plot_spacer() +
  legend + 
  plot_layout(ncol = 1, heights = c(0.1, -0.1, 4, -0.1, 1))

final_plot

## Find Markers

In [ ]:
# Find all markers for each cluster in the Seurat object
# find_markers <- FindAllMarkers(normalized_seurat, 
#                           logfc.threshold = 0.25,
#                           test.use = "wilcox",
#                           assay = "SCT",
#                           min.pct = 0.25,
#                           slot = "data") 

# # Filter and sort the find_markers
# find_markers <- find_markers %>%  
#     mutate(cluster = as.integer(as.character(cluster)), na.rm=TRUE) %>% 
#     select(cluster, gene, avg_log2FC, p_val, p_val_adj, pct.1, pct.2) %>%
#     dplyr::filter(p_val_adj < 0.05) %>%
#     dplyr::filter(abs(avg_log2FC) > 0.25) %>%
#     arrange(cluster, desc(avg_log2FC)) %>% 
#     mutate_if(is.numeric,round, 3) %>%
#     group_by(cluster) 


# write.csv((find_markers %>% top_n(n=5, wt=avg_log2FC)), "./mouse_NTN/reports/markers_clustering_top5.csv", row.names = FALSE)
# write.csv((find_markers %>% top_n(n=100, wt=avg_log2FC)), "./mouse_NTN/reports/markers_clustering_top100.csv", row.names = FALSE)

In [ ]:
file_path <- "/mouse_NTN/reports/ncells.xlsx"

df <-  normalized_seurat@meta.data %>% group_by(alias_id, Treatment) %>% count()
dff <- normalized_seurat@meta.data %>% group_by(clustering_harmony) %>% count()
df2 <- normalized_seurat@meta.data %>% group_by(clustering_harmony, Treatment) %>% count()
df3 <- normalized_seurat@meta.data %>% group_by(clustering_harmony, alias_id) %>% count()

In [ ]:
normalized_seurat <- RenameIdents(object = normalized_seurat, 
                                 "0" = "Early_PT-PCT",
                                 "1" = "LOH",
                                 "2" = "PTS3",
                                 "3" = "Collecting_duct, IC",
                                 "4" = "Macrophagues",
                                 "5" = "EC",
                                 "6" = "DCT, Collecting_duct",
                                 "7" = "PC",
                                 "8" = "FIB",
                                 "9" = "TDLH",
                                 "10" = "Adipocytes",
                                 "11" = "POD",
                                 "12" = "Urothelium")

normalized_seurat$first_annotation <- Idents(normalized_seurat)

DimPlot(normalized_seurat, label=T, group.by = "first_annotation", label.size = 6, repel=T) + NoLegend()  + theme(axis.title.x=element_blank())

## Some reflexions

In [ ]:
options(repr.plot.width = 26, repr.plot.height = 10, res=1200)
p <- DimPlot(normalized_seurat, label=T, group.by = "first_annotation", label.size = 6, repel=T) + NoLegend()  + theme(axis.title.x=element_blank())
p1 <- DimPlot(normalized_seurat, label=T, group.by = "SCT_snn_res.0.2", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text =  element_blank()) + theme(axis.title.y = element_blank())
p2 <- DimPlot(normalized_seurat, label=T, group.by = "SCT_snn_res.0.3", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
p3 <- DimPlot(normalized_seurat, label=T, group.by = "SCT_snn_res.0.4", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
wrap_plots(p,p1,p2, p3, ncol=4) & plot_layout(guides="collect")

In [154]:
normalized_seurat$second_annotation <- normalized_seurat$first_annotation %>% as.character()

### Dividing PT

In [ ]:
PTS1 <- c("Slc5a12", "Slc13a3", "Slc22a6", "Prodh2", "Slc5a2", "Slc22a8")
PTS2 <- c("Slc5a12", "Slc13a3", "Slc22a6", "Slc34a1", "Slc22a7")
PTS3 <- c("Slc22a7", "Mogat1", "Slc5a11", "Slc7a13", "Slc5a8", "Abcc3", "Satb2")

p1 <- DotPlot(normalized_seurat, features= PTS1, group.by= "SCT_snn_res.0.3") + ggtitle("PTS1") + NoLegend()
p2 <- DotPlot(normalized_seurat, features= PTS2, group.by= "SCT_snn_res.0.3") + ggtitle("PTS2") + NoLegend()
p3 <- DotPlot(normalized_seurat, features= PTS3, group.by= "SCT_snn_res.0.3") + ggtitle("PTS3") + NoLegend()
wrap_plots(p1,p2,p3)

In res0.3, clearly 0 is PTS3, let's go further in genes that are not common

In [ ]:
PTS1 <- c("Prodh2", "Slc5a2", "Slc22a8")
PTS2 <- c("Slc34a1")

p1 <- DotPlot(normalized_seurat, features= PTS1, group.by= "SCT_snn_res.0.3") + ggtitle("PTS1") + NoLegend()
p2 <- DotPlot(normalized_seurat, features= PTS2, group.by= "SCT_snn_res.0.3") + ggtitle("PTS2") + NoLegend()
wrap_plots(p1,p2)

Slc5a2 is the key for distinguish between S1 and S2

In [157]:
PTS1s <- normalized_seurat@meta.data %>% filter(SCT_snn_res.0.3 =="3") %>% rownames()
normalized_seurat@meta.data[PTS1s,"second_annotation"] <- "PTS1"

PTS2s <- normalized_seurat@meta.data %>% filter(SCT_snn_res.0.3 =="1") %>% rownames()
normalized_seurat@meta.data[PTS2s,"second_annotation"] <- "PTS2"

In [158]:
#remove few cells didn't stay cluster
outliers <- subset(x = normalized_seurat,subset = second_annotation == "Early_PT-PCT")
# p1 <- DotPlot(outliers, features= PTS1, group.by= "second_annotation") + ggtitle("PTS1") + NoLegend()
# p2 <- DotPlot(outliers, features= PTS2, group.by= "second_annotation") + ggtitle("PTS2") + NoLegend()
# wrap_plots(p1,p2)

In [159]:
outliers <- normalized_seurat@meta.data %>% filter(second_annotation == "Early_PT-PCT") %>% rownames()
normalized_seurat@meta.data[outliers,"second_annotation"] <- "PTS1"

In [ ]:
p <- DimPlot(normalized_seurat, label=T, group.by = "first_annotation", label.size = 6, repel=T) + NoLegend()  + theme(axis.title.x=element_blank())
p1 <- DimPlot(normalized_seurat, label=T, group.by = "SCT_snn_res.0.2", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text =  element_blank()) + theme(axis.title.y = element_blank())
p2 <- DimPlot(normalized_seurat, label=T, group.by = "SCT_snn_res.0.3", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
p3 <- DimPlot(normalized_seurat, label=T, group.by = "second_annotation", label.size = 6, repel=T) + theme(axis.title.x=element_blank())
wrap_plots(p,p1,p2, p3, ncol=4) & plot_layout(guides="collect")

### Macrophagues and B cells

In [ ]:
macrophagues <- markers_long %>% filter(cell_type== "Macro" & marker != "NA") %>% pull(marker)
Bcells <- c("Igha", "Igkc")
p <- DotPlot(normalized_seurat, features= macrophagues, group.by= "SCT_snn_res.0.3") + ggtitle("Macrophagues") + NoLegend()
p2 <- DotPlot(normalized_seurat, features= Bcells, group.by= "SCT_snn_res.0.3") + ggtitle("B cells") + NoLegend()
wrap_plots(p,p2)

In [162]:
Bcells_ids <- normalized_seurat@meta.data %>% filter(SCT_snn_res.0.3 =="10") %>% rownames()
normalized_seurat@meta.data[Bcells_ids,"second_annotation"] <- "Bcell"

In [ ]:
p <- DimPlot(normalized_seurat, label=T, group.by = "first_annotation", label.size = 6, repel=T) + NoLegend()  + theme(axis.title.x=element_blank())
p1 <- DimPlot(normalized_seurat, label=T, group.by = "SCT_snn_res.0.2", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text =  element_blank()) + theme(axis.title.y = element_blank())
p2 <- DimPlot(normalized_seurat, label=T, group.by = "SCT_snn_res.0.3", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
p3 <- DimPlot(normalized_seurat, label=T, group.by = "second_annotation", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
wrap_plots(p,p1,p2, p3, ncol=4) & plot_layout(guides="collect")

### Duct_IC and DCT_collecting duct

In [ ]:
p <- DimPlot(normalized_seurat, label=T, group.by = "first_annotation", label.size = 6, repel=T) + NoLegend()  + theme(axis.title.x=element_blank())
p1 <- DimPlot(normalized_seurat, label=T, group.by = "SCT_snn_res.0.2", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text =  element_blank()) + theme(axis.title.y = element_blank())
p2 <- DimPlot(normalized_seurat, label=T, group.by = "SCT_snn_res.0.3", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
p3 <- DimPlot(normalized_seurat, label=T, group.by = "second_annotation", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
p4 <- DimPlot(normalized_seurat, label=T, group.by = "SCT_snn_res.0.4", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
wrap_plots(p,p1,p2, p3, p4, ncol=5) & plot_layout(guides="collect")

In [ ]:
ICs <- markers_long %>% filter(cell_type== "IC" & marker != "NA") %>% pull(marker)
DCTs <- markers_long %>% filter(cell_type== "DCT" & marker != "NA") %>% pull(marker)

p <- DotPlot(normalized_seurat, features= ICs, group.by= "SCT_snn_res.0.4") + ggtitle("ICs") + NoLegend()
p2 <- DotPlot(normalized_seurat, features= DCTs, group.by= "SCT_snn_res.0.4") + ggtitle("DCTs") + NoLegend()
wrap_plots(p,p2)

Instead of res.0.3, is better seen in 0.4 So in the res.0.4 number 10 is Intercalated cells. But what is the difference between 7 and 5

In [166]:
IC_ids <- normalized_seurat@meta.data %>% filter(SCT_snn_res.0.4 =="10") %>% rownames()
normalized_seurat@meta.data[IC_ids,"second_annotation"] <- "IC"

In [ ]:
p <- DimPlot(normalized_seurat, label=T, group.by = "first_annotation", label.size = 6, repel=T) + NoLegend()  + theme(axis.title.x=element_blank())
p1 <- DimPlot(normalized_seurat, label=T, group.by = "SCT_snn_res.0.2", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text =  element_blank()) + theme(axis.title.y = element_blank())
p2 <- DimPlot(normalized_seurat, label=T, group.by = "SCT_snn_res.0.3", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
p3 <- DimPlot(normalized_seurat, label=T, group.by = "second_annotation", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
p4 <- DimPlot(normalized_seurat, label=T, group.by = "SCT_snn_res.0.4", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
wrap_plots(p,p1,p2, p3, p4, ncol=5) & plot_layout(guides="collect")

Let's go back to KPMP to see the markers: (\data\KPMP_Markers.xslx )
* DCT: "Slc12a3", "Cnnm2", "Fgf13", "Klhl3", "Lhx1", "Trpm6"
* Collecting tubule: "Slc8a1", "Scn2a", "Hsd11b2", "Calb1"


In [ ]:
DCTs <- c("Slc12a3", "Cnnm2", "Fgf13", "Klhl3", "Lhx1", "Trpm6")
CNT <- c("Slc8a1", "Scn2a", "Hsd11b2", "Calb1")

p <- DotPlot(normalized_seurat, features= DCTs, group.by= "SCT_snn_res.0.4") + ggtitle("DCTs") + NoLegend()
p2 <- DotPlot(normalized_seurat, features= CNT, group.by= "SCT_snn_res.0.4") + ggtitle("CNT") + NoLegend()
wrap_plots(p,p2)

In [169]:
DCT_ids <- normalized_seurat@meta.data %>% filter(SCT_snn_res.0.4 =="7") %>% rownames()
normalized_seurat@meta.data[DCT_ids,"second_annotation"] <- "DCT"

CNT_ids <- normalized_seurat@meta.data %>% filter(SCT_snn_res.0.4 =="5") %>% rownames()
normalized_seurat@meta.data[CNT_ids,"second_annotation"] <- "CNT"

In [170]:
outlier <- normalized_seurat@meta.data %>% filter(second_annotation == "Collecting_duct, IC") %>% rownames()
normalized_seurat@meta.data[outlier,"second_annotation"] <- "IC"

In [ ]:
p <- DimPlot(normalized_seurat, label=T, group.by = "first_annotation", label.size = 6, repel=T) + NoLegend()  + theme(axis.title.x=element_blank())
p1 <- DimPlot(normalized_seurat, label=T, group.by = "SCT_snn_res.0.2", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text =  element_blank()) + theme(axis.title.y = element_blank())
p2 <- DimPlot(normalized_seurat, label=T, group.by = "SCT_snn_res.0.3", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
p3 <- DimPlot(normalized_seurat, label=T, group.by = "second_annotation", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
wrap_plots(p,p1,p2, p3, ncol=4) & plot_layout(guides="collect")

### Find VSMC in EC

In [ ]:
Endos <- markers_long %>% filter(cell_type== "Endo" & marker != "NA") %>% pull(marker)
VSCMs <- c("Acta2", "Myh11")
p <- DotPlot(normalized_seurat, features= Endos, group.by= "SCT_snn_res.0.5") + ggtitle("Endos") + NoLegend()
p2 <- DotPlot(normalized_seurat, features= VSCMs, group.by= "SCT_snn_res.0.5") + ggtitle("VSCMs") + NoLegend()
wrap_plots(p,p2)

In [173]:
EC_ids <- normalized_seurat@meta.data %>% filter(SCT_snn_res.0.5 =="9") %>% rownames()
normalized_seurat@meta.data[EC_ids,"second_annotation"] <- "EC"

VSCM_ids <- normalized_seurat@meta.data %>% filter(SCT_snn_res.0.5 =="18") %>% rownames()
normalized_seurat@meta.data[VSCM_ids,"second_annotation"] <- "VSCM"

In [174]:
normalized_seurat$second_annotation <- as.factor(normalized_seurat$second_annotation)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 10, res=1200)
DimPlot(normalized_seurat, label=T, group.by = "second_annotation", label.size = 6, repel=T) + NoLegend()

## Third annotation

In [176]:
normalized_seurat$third_annotation <- normalized_seurat$second_annotation %>% as.character()

### Separate LOH

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 10, res=1200)
p1 <- DimPlot(normalized_seurat, label=T, group.by = "third_annotation", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank())
p2 <- DimPlot(normalized_seurat, label=T, group.by = "SCT_snn_res.0.3", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
p4 <- DimPlot(normalized_seurat, label=T, group.by = "SCT_snn_res.0.4", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
wrap_plots(p1,p2, p4, ncol=3) & plot_layout(guides="collect")

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 15, res=1200)
Thin_Limb_Cell <- c("Cryab", "Tacstd2", "Klrg2", "Col26a1", "Boc")
Descending_Thin_Limb_Cell_Type_2 <- c("Vcam1", "Slc39a8", "Aqp1", "Lrrc4c", "Satb2")
Ascending_Thin_Limb_Cell <- c("Bcas1", "Clcnka", "Cldn10", "Prox1")

Thick_Ascending_Limb_Cell <- c("Casr", "Slc12a1", "Umod")
Medullary_Thick_Ascending_Limb_Cell <- c("Esrrb", "Egf", "Cldn14", "Mfsd4a", "Kctd16", "Rap1gap", "Ank2", "Cyfip2")
Cortical_Thick_Ascending_Limb_Cell <- c("Ppm1e", "Gp2", "Enox1", "Tmem207", "Tmem52b", "Cldn16", "Wnk1")

p1 <- DotPlot(normalized_seurat, features = Thin_Limb_Cell, group.by = "SCT_snn_res.0.4") + ggtitle("Thin Limb Cell") + NoLegend()
p2<- DotPlot(normalized_seurat, features = Descending_Thin_Limb_Cell_Type_2, group.by = "SCT_snn_res.0.4") + ggtitle("Descending Thin Limb Cell Type 2") + NoLegend()
p5 <- DotPlot(normalized_seurat, features = Ascending_Thin_Limb_Cell, group.by = "SCT_snn_res.0.4") + ggtitle("Ascending Thin Limb Cell") + NoLegend()
p6 <- DotPlot(normalized_seurat, features = Thick_Ascending_Limb_Cell, group.by = "SCT_snn_res.0.4") + ggtitle("Thick Ascending Limb Cell") + NoLegend()
p7 <- DotPlot(normalized_seurat, features = Medullary_Thick_Ascending_Limb_Cell, group.by = "SCT_snn_res.0.4") + ggtitle("Medullary Thick Ascending Limb Cell") + NoLegend()
p8 <- DotPlot(normalized_seurat, features = Cortical_Thick_Ascending_Limb_Cell, group.by = "SCT_snn_res.0.4") + ggtitle("Cortical Thick Ascending Limb Cell") + NoLegend()
wrap_plots(p1,p2,p5,p6,p7,p8, ncol=3)

In [179]:
options(repr.plot.width = 8, repr.plot.height = 7, res=1200)
Thin_L <- normalized_seurat@meta.data %>% filter(SCT_snn_res.0.4 =="9"  | SCT_snn_res.0.4 =="14") %>% rownames()
normalized_seurat@meta.data[Thin_L,"third_annotation"] <- "ThinL"

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 7, res=1200)
Thick_L <- normalized_seurat@meta.data %>% filter(SCT_snn_res.0.4 =="3"  | SCT_snn_res.0.4 =="4") %>% rownames()
normalized_seurat@meta.data[Thick_L,"third_annotation"] <- "ThickL"
p2 <- DimPlot(normalized_seurat, label=T, group.by = "third_annotation", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
p2

In [ ]:
outlier <- normalized_seurat@meta.data %>% filter(third_annotation == "LOH") %>% rownames()
normalized_seurat@meta.data[outlier,"third_annotation"] <- "ThinL"
p2 <- DimPlot(normalized_seurat, label=T, group.by = "third_annotation", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
p2

### What are these small clusters? IMM?

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 9, res=1200)
p <- DimPlot(normalized_seurat, label=T, group.by = "SCT_snn_res.0.4", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
p2 <- DimPlot(normalized_seurat, label=T, group.by = "third_annotation", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
wrap_plots(p2,p)

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 15, res=1200)
BCell<- c("Bank1", "Blk", "Ms4a1", "Bach2")
PlasmaCell<- c("Igkc", "Tent5c", "Mzb1", "Cd38", "Jchain")
TCell <- c("Cd96", "Cd247", "Themis", "Bcl11b", "Camk4", "Il7r")
macrophagues <- markers_long %>% filter(cell_type== "Macro" & marker != "NA") %>% pull(marker)
Naturalkiller <- c("Runx3", "Nkg7", "Ccl5","Ccl4", "Gzma")
Dendritic_Cell <- c("Itgax",  "Csf2ra", "Ciita", "Wdfy4", "Flt3", "Cadm1", "Zbtb46", "Clec9a")
Non_Classical_monocyte <- c("Ctss", "Irak3", "Tcf7l2", "Tnfrsf1b")
Neutrophil <- c("S100a9", "S100a8")

p <- DotPlot(normalized_seurat, features= TCell, group.by= "SCT_snn_res.0.4") + ggtitle("TCell") + NoLegend()
p3 <- DotPlot(normalized_seurat, features= BCell, group.by= "SCT_snn_res.0.4") + ggtitle("BCell") + NoLegend()
p2 <- DotPlot(normalized_seurat, features= PlasmaCell, group.by= "SCT_snn_res.0.4") + ggtitle("PlasmaCell") + NoLegend()
p6 <- DotPlot(normalized_seurat, features= Naturalkiller, group.by= "SCT_snn_res.0.4") + ggtitle("Naturalkiller") + NoLegend()
p4 <- DotPlot(normalized_seurat, features= macrophagues, group.by= "SCT_snn_res.0.4") + ggtitle("macrophagues") + NoLegend()
p5 <- DotPlot(normalized_seurat, features= Dendritic_Cell, group.by= "SCT_snn_res.0.4") + ggtitle("Dendritic_Cell") + NoLegend()
p7 <- DotPlot(normalized_seurat, features= Non_Classical_monocyte, group.by= "SCT_snn_res.0.4") + ggtitle("Non_Classical_monocyte") + NoLegend()
p8 <- DotPlot(normalized_seurat, features= Neutrophil, group.by= "SCT_snn_res.0.4") + ggtitle("Neutrophil") + NoLegend()


wrap_plots(p, p3,p2,p4, p6,p5,p7,p8, ncol=4)

### Other small cluster, FIB?

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 7, res=1200)
DimPlot(normalized_seurat, label=T, group.by = "SCT_snn_res.0.8", label.size = 6, repel=T) + NoLegend() 

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 13, res=1200)
FIB <- subset(x = normalized_seurat,subset = second_annotation == "FIB")
# DimPlot(FIB, label=T, group.by = "SCT_snn_res.0.4", label.size = 6, repel=T) + NoLegend() 
FIB <- FIB %>% FindNeighbors(., dims = 1:50) %>% 
                    FindClusters(., resolution  = c(0.1) )

FIB$SCT_snn_res.0.1 %>% table()
p <- DimPlot(FIB, label=T, group.by = "SCT_snn_res.0.1", label.size = 6, repel=T) + NoLegend() 
p2 <- DimPlot(FIB, label=F, group.by = "Treatment") 
wrap_plots(p,p2)

In [ ]:
Idents(FIB) <- "SCT_snn_res.0.1"
markers <- FindMarkers(FIB, ident.1 = "1", ident.2="0") %>% filter((p_val_adj < 0.05) & (avg_log2FC > 0.25)) %>% arrange(desc(avg_log2FC)) %>% head(100)
markers %>% head() 

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 10, res=1200)
p1 <- DimPlot(normalized_seurat, label=T, group.by = "first_annotation", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank())
p2 <- DimPlot(normalized_seurat, label=T, group.by = "second_annotation", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
p4 <- DimPlot(normalized_seurat, label=T, group.by = "third_annotation", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
wrap_plots(p1,p2, p4, ncol=3) & plot_layout(guides="collect")

After discussion with biologists, for now we are going to split LOH only in thin and thick, and merge Bcell and Macrophagues since is quite difficult to separate them.

In [188]:
normalized_seurat$annotation <- normalized_seurat$third_annotation
IMM_ids <- normalized_seurat@meta.data %>% filter(annotation =="Bcell" | annotation == "Macrophagues") %>% rownames()
normalized_seurat@meta.data[IMM_ids,"annotation"] <- "IMM"

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 10, res=1200)
p1 <- DimPlot(normalized_seurat, label=T, group.by = "annotation", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank())
p2 <- DimPlot(normalized_seurat, group.by = "Treatment") + theme(axis.title.x=element_blank(), axis.text = element_blank()) + theme(axis.title.y = element_blank())
wrap_plots(p1,p2,ncol=2) & plot_layout(guides="collect")

# Contribution to each condition to the clusters

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 12)
create_barplot <- function(column_name, seurat_object) {
    df <- seurat_object@meta.data %>%
        group_by(!!sym(column_name), annotation) %>%
        summarise(n = n(), .groups = "drop") %>%
        group_by(!!sym(column_name)) %>%
        mutate(freq = n / sum(n))
      
      # Create barplot
      p <- ggplot(df, aes_string(x = "annotation", y = "freq", fill = column_name)) +
        geom_bar(stat = "identity", position = "fill") +
        labs(x = "Cell type", y = "Normalized Percentage", fill = column_name) +
        theme_minimal() +
        scale_y_continuous(labels = scales::percent) +
        theme(
          legend.position = "right",
          axis.title.y = element_blank(),
          axis.text.y = element_blank(),
          axis.ticks.y = element_blank(),
          axis.text.x = element_text(angle = 45, vjust = 1, hjust = 1, size = 12),
          panel.border = element_blank()
        )
      return(p)
}

# Vector of column names to create plots for
columns_to_plot <- c("alias_id", "Treatment")

# Use lapply to create a list of plots
list_of_plots <- lapply(columns_to_plot, create_barplot, seurat_object = normalized_seurat)

p <- wrap_plots(list_of_plots, ncol=1)
plot(p)

if (write_plots) {
    ggsave(p, filename = file.path(plot_dir, "/plots/analysis_scRNAseq/Contribution_sample_treatment.png"), width = 15, height = 12)
}

In [98]:
options(repr.plot.width = 12, repr.plot.height = 8)
p <- DimPlot(normalized_seurat, label=T, group.by = "annotation", label.size = 6, repel=T) + NoLegend() + theme(axis.title.x=element_blank())

if (write_plots) {
    ggsave(p, filename = file.path(plot_dir, "/plots/analysis_scRNAseq/UMAP_annotated.png"), width = 12, height = 8)
}